In [1]:
import os
import re
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Function to extract metrics from the last line of the train.txt file
def extract_metrics(file_path):
    with open(file_path, 'r') as file:
        last_line = file.readlines()[-1].strip()
    
    # Use regex to extract PSNR and MS-SSIM values
    psnr_match = re.search(r'PSNR:([0-9]+\.[0-9]+)', last_line)
    msssim_match = re.search(r'MS-SSIM:([0-9]+\.[0-9]+)', last_line)
    
    psnr = float(psnr_match.group(1)) if psnr_match else None
    msssim = float(msssim_match.group(1)) if msssim_match else None
    return psnr, msssim

In [3]:
def gather_data(base_path, modelsize_path='../modelsize'):
    target_gaussian_numbers = {10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000}
    gaussian_numbers = []
    psnr_values = []
    msssim_values = []
    model_size = []  # List to store model sizes in MB
    
    # Iterate through each folder in the base path
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            try:
                # Extract the Gaussian number from the folder name
                gaussian_number = int(folder_name.split('_')[-1])
                
                # Only process if the Gaussian number is in the target list
                if gaussian_number in target_gaussian_numbers:
                    train_file = os.path.join(folder_path, 'train.txt')

                    if os.path.exists(train_file):
                        psnr, msssim = extract_metrics(train_file)
                        gaussian_numbers.append(gaussian_number)
                        psnr_values.append(psnr)
                        msssim_values.append(msssim)
            except Exception as e:
                print(f"Error processing folder {folder_name}: {e}")

    # Read and process the model size CSV files
    csv_files = ['Beauty_gaussian_model_sizes.csv', 'HoneyBee_gaussian_model_sizes.csv', 'Jockey_gaussian_model_sizes.csv']
    for csv_file in csv_files:
        file_path = os.path.join(modelsize_path, csv_file)
        
        # Read the CSV file and store the results
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)

            # Sort the dataframe by 'Gaussian_Count' in ascending order
            df = df.sort_values(by='Gaussian_Count')

            for _, row in df.iterrows():
                gaussian_number = row['Gaussian_Count']
                file_size_bytes = row['File_Size_Bytes']
                
                if gaussian_number in target_gaussian_numbers:
                    # Convert file size from bytes to MB (1 MB = 1,048,576 bytes)
                    file_size_mb = file_size_bytes / (1024 * 1024)
                    model_size.append(file_size_mb)
                    print(gaussian_number,file_size_mb)
                    
        else:
            print(f"CSV file not found: {csv_file}")

    return gaussian_numbers, psnr_values, msssim_values, model_size

In [4]:
def plot_metrics(model_size, gaussian_numbers, psnr_values, msssim_values, video_name, save_directory):
    # Create directory if it doesn't exist
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    plt.figure(figsize=(10, 5))

    # PSNR Plot
    plt.subplot(1, 2, 1)
    
    # Plot with circle sizes based on model_size, transparent circles with solid cores
    plt.scatter(gaussian_numbers, psnr_values, s=[ms * 10 for ms in model_size], alpha=0.5, edgecolor='black', facecolor='none', label='Model Size')
    
    plt.scatter(gaussian_numbers, psnr_values, s=20, color='blue', label='PSNR')  # Core solid points
    
    plt.xlabel('Number of Gaussians')
    plt.ylabel('PSNR')
    plt.title(f'PSNR vs Gaussians for {video_name}')
    
    # MS-SSIM Plot
    plt.subplot(1, 2, 2)
    
    # Plot with circle sizes based on model_size, transparent circles with solid cores
    plt.scatter(gaussian_numbers, msssim_values, s=[ms * 10 for ms in model_size], alpha=0.5, edgecolor='black', facecolor='none', label='Model Size')
    plt.scatter(gaussian_numbers, msssim_values, s=20, color='orange', label='MS-SSIM')  # Core solid points
    
    plt.xlabel('Number of Gaussians')
    plt.ylabel('MS-SSIM')
    plt.title(f'MS-SSIM vs Gaussians for {video_name}')
    
    plt.tight_layout()
    
    # Save the plot to the specified directory with a proper filename
    save_path = os.path.join(save_directory, f'{video_name}_metrics_plot.png')
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory

In [5]:
# Function to plot PSNR and save the plot
def plot_psnr(model_size, gaussian_numbers, psnr_values, video_name, save_directory):
    # Create directory if it doesn't exist
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    plt.figure(figsize=(5, 5))

    # Plot with circle sizes based on model_size, transparent circles with solid cores
    plt.scatter(gaussian_numbers, psnr_values, s=[ms * 10 for ms in model_size], alpha=0.5, edgecolor='black', facecolor='none', label='Model Size')
    plt.scatter(gaussian_numbers, psnr_values, s=20, color='blue', label='PSNR')  # Core solid points

    plt.xlabel('Number of Gaussians')
    plt.ylabel('PSNR')
    plt.title(f'PSNR vs Gaussians for {video_name}')
    plt.tight_layout()

    # Save the PSNR plot to the specified directory
    save_path = os.path.join(save_directory, f'{video_name}_PSNR_plot.png')
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory

# Function to plot MS-SSIM and save the plot
def plot_msssim(model_size, gaussian_numbers, msssim_values, video_name, save_directory):
    # Create directory if it doesn't exist
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    plt.figure(figsize=(5, 5))

    # Plot with circle sizes based on model_size, transparent circles with solid cores
    plt.scatter(gaussian_numbers, msssim_values, s=[ms * 10 for ms in model_size], alpha=0.5, edgecolor='black', facecolor='none', label='Model Size')
    plt.scatter(gaussian_numbers, msssim_values, s=20, color='orange', label='MS-SSIM')  # Core solid points

    plt.xlabel('Number of Gaussians')
    plt.ylabel('MS-SSIM')
    plt.title(f'MS-SSIM vs Gaussians for {video_name}')
    plt.tight_layout()

    # Save the MS-SSIM plot to the specified directory
    save_path = os.path.join(save_directory, f'{video_name}_MS-SSIM_plot.png')
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory

In [6]:
# Function to plot PSNR for all videos with circle size based on model size
def plot_psnr_all(all_data, save_directory):
    plt.figure(figsize=(8, 6))

    # Iterate through each video's data and plot PSNR with circle size based on model_size
    for video_name, data in all_data.items():
        model_size = data['model_size']
        gaussian_numbers = data['gaussian_numbers']
        psnr_values = data['psnr_values']
        
        # Plot with circle sizes based on model_size, transparent circles with solid cores
        plt.scatter(gaussian_numbers, psnr_values, s=[ms * 10 for ms in model_size], alpha=0.5, edgecolor='black', facecolor='none', label=video_name)
        plt.scatter(gaussian_numbers, psnr_values, s=20, label=f'{video_name} Core')  # Core solid points

    plt.xlabel('Number of Gaussians')
    plt.ylabel('PSNR')
    plt.title('PSNR vs Gaussians for All Videos')
    plt.legend()  # Show legend to differentiate between videos
    plt.tight_layout()

    # Save the PSNR plot to the specified directory
    save_path = os.path.join(save_directory, 'All_Videos_PSNR_plot.png')
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory


# Function to plot MS-SSIM for all videos with circle size based on model size
def plot_msssim_all(all_data, save_directory):
    plt.figure(figsize=(8, 6))

    # Iterate through each video's data and plot MS-SSIM with circle size based on model_size
    for video_name, data in all_data.items():
        model_size = data['model_size']
        gaussian_numbers = data['gaussian_numbers']
        msssim_values = data['msssim_values']
        
        # Plot with circle sizes based on model_size, transparent circles with solid cores
        plt.scatter(gaussian_numbers, msssim_values, s=[ms * 10 for ms in model_size], alpha=0.5, edgecolor='black', facecolor='none', label=video_name)
        plt.scatter(gaussian_numbers, msssim_values, s=20, label=f'{video_name} Core')  # Core solid points

    plt.xlabel('Number of Gaussians')
    plt.ylabel('MS-SSIM')
    plt.title('MS-SSIM vs Gaussians for All Videos')
    plt.legend()  # Show legend to differentiate between videos
    plt.tight_layout()

    # Save the MS-SSIM plot to the specified directory
    save_path = os.path.join(save_directory, 'All_Videos_MS-SSIM_plot.png')
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory



In [7]:
# Main code execution
base_folder =  '../result'  # Set this to the base folder path containing 'Beauty', 'Honeybee', 'Jockey'
save_directory = './img' # Set the directory where you want to save the plots
all_data = {}
for video_folder in ['Beauty', 'Honeybee', 'Jockey']:
    video_path = base_folder + '/'+ video_folder
    gaussian_numbers, psnr_values, msssim_values,model_size = gather_data(video_path)
    
    # Sort values by Gaussian numbers for plotting
    sorted_indices = sorted(range(len(gaussian_numbers)), key=lambda k: gaussian_numbers[k])
    gaussian_numbers = [gaussian_numbers[i] for i in sorted_indices]
    psnr_values = [psnr_values[i] for i in sorted_indices]
    msssim_values = [msssim_values[i] for i in sorted_indices]
    model_size = [model_size[i] for i in sorted_indices]
    # Store the data for each video in the all_data dictionary
    all_data[video_folder] = {
        'gaussian_numbers': gaussian_numbers,
        'psnr_values': psnr_values,
        'msssim_values': msssim_values,
        'model_size':model_size
    }

    # Plot PSNR and MS-SSIM for each video folder and save the image
    plot_metrics(model_size,gaussian_numbers, psnr_values, msssim_values, video_folder, save_directory)
    plot_psnr(model_size,gaussian_numbers, psnr_values, video_folder, save_directory)
    plot_msssim(model_size,gaussian_numbers, msssim_values, video_folder, save_directory)

# Plot and save combined PSNR and MS-SSIM for all videos
plot_psnr_all(all_data, save_directory)
plot_msssim_all(all_data, save_directory)


10000 207.21173667907715
15000 310.15363121032715
20000 413.20538902282715
25000 516.1496028900146
30000 619.2013607025146
35000 722.1443538665771
40000 825.1961116790771
45000 928.1380062103271
50000 1031.1897640228271
55000 1134.1316585540771
60000 1237.1834163665771
65000 1340.1253108978271
70000 1443.1828060150146
10000 207.21173667907715
15000 310.15363121032715
20000 413.20538902282715
25000 516.1496028900146
30000 619.2013607025146
35000 722.1443538665771
40000 825.1961116790771
50000 1031.1897640228271
55000 1134.1316585540771
60000 1237.1834163665771
65000 1340.1253108978271
70000 1443.1828060150146
10000 207.21173667907715
15000 310.15363121032715
20000 413.20538902282715
25000 516.1496028900146
30000 619.2013607025146
35000 722.1443538665771
40000 825.1961116790771
45000 928.1380062103271
50000 1031.1897640228271
55000 1134.1316585540771
60000 1237.1834163665771
65000 1340.1253108978271
70000 1443.1828060150146
10000 207.21173667907715
15000 310.15363121032715
20000 413.2053